# Ejercicio 12: Web Scraping

    
    


**Estudiante:** José Quiros



## Objetivo de la práctica

El objetivo de este ejercicio es construir un web scraper que recoja datos de un website.

### Parte 0: Planificar
1. Identificar los datos que quieres obtener.
2. Elegir el sitio web objetivo.
3. Planificar la estructura del corpus.



1. **Datos:** título, valoración, porciones, tiempo, factores nutricionales, descripción, imagen, ingredientes, pasos.
2. **Web:** Allrecipes.com
  *   Partiendo de: https://www.allrecipes.com/recipe/83557/juicy-roasted-chicken/

3. Usar una URL base e ir saltando a través de las recetas recomendadas para generar el corpus. A su vez, el corpus debe tener una longitud de 100 recetas y cada documento debe estar formado por los datos recuperados.




## Parte 1: Entender el sitio web objetivo

- Analizar la estructura de la página web a ser analizada.
- Identificar los elementos HTML que contienen los datos bsuscados.

In [1]:
# Función para recuperar el contenido de una URL, usando subprocess y almacenarlo en una cadena
import subprocess
from bs4 import BeautifulSoup
def recuperar_receta(url):
  comando_curl = ["curl", url]
  resultado = subprocess.run(comando_curl, capture_output=True, text=True, check=True)
  return resultado.stdout

In [2]:
!curl https://www.allrecipes.com/recipe/221964/spicy-pbj-wings/ > pag.html

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  507k    0  507k    0     0  1041k      0 --:--:-- --:--:-- --:--:-- 1040k


In [3]:
!wget https://www.allrecipes.com/recipe/221964/spicy-pbj-wings/

--2025-07-23 21:24:06--  https://www.allrecipes.com/recipe/221964/spicy-pbj-wings/
Resolving www.allrecipes.com (www.allrecipes.com)... 162.159.141.224, 172.66.1.220, 2606:4700:7::1d8, ...
Connecting to www.allrecipes.com (www.allrecipes.com)|162.159.141.224|:443... connected.
HTTP request sent, awaiting response... 460 
2025-07-23 21:24:06 ERROR 460: (no description).



In [7]:
from bs4 import BeautifulSoup
file = "pag.html"

with open(file, "r", encoding="utf-8") as file:
  html_content = file.read()

print(html_content[:300])

<!DOCTYPE html>
<html id="recipeScTemplate_1-0" class="comp recipeScTemplate html mntl-html no-js taxlevel-1 " data-mm-ads-resource-version="2.2.20" data-ddm-standard-video="true" data-mm-video-resource-version="3.0.5" data-mm-myrecipes-resource-version="3.1.10" data-mantle-resource-version="4.2.74"


In [5]:
soup = BeautifulSoup(html_content, "html.parser")

In [6]:
# Título
def obtener_titulo(soup):
  return soup.find("meta", {"property":"og:title"})["content"]

title = soup.find("meta", {"property":"og:title"})["content"]
title

'Spicy PBJ Wings'

In [10]:
# Valoración
# En la página que no tenga una valoración se agrega valor por defecto "sin valoración"
def obtener_valoracion (soup):
  item = soup.find("div", {'id':"mm-recipes-review-bar__rating_1-0"})
  return item.get_text(strip=True) if item else "sin valoración"

valoracion = soup.find("div", {'id':"mm-recipes-review-bar__rating_1-0"})
valoracion.text

'4.5'

In [9]:
# Porciones
# La página puede no tener este dato, por ende, se envia por defecto una cadena vacía
def obtener_porciones(soup):
  servings = ""

  for item in soup.select('div.mm-recipes-details__item'):
    label = item.select_one('div.mm-recipes-details__label')
    value = item.select_one('div.mm-recipes-details__value')

    if label and value and "Servings:" in label.get_text(strip=True):
          servings = value.get_text(strip=True)
          break # Exit loop once servings are found

  return servings



items = soup.find_all('div', class_='mm-recipes-details__item')
servings = ""
for item in items:
   label = item.find('div', class_='mm-recipes-details__label')
   value = item.find('div', class_='mm-recipes-details__value')

   if label and value and "Servings:" in label.text:
        servings = value.text.strip()
        break # Exit loop once servings are found

print(f"Porciones: {servings}")

Porciones: 8


In [11]:
# Tiempo total
# La página puede no tener este dato, por ende, se envia por defecto una cadena vacía
def obtener_tiempo(soup):
  tiempo = ""

  for item in soup.select('div.mm-recipes-details__item'):
    label = item.select_one('div.mm-recipes-details__label')
    value = item.select_one('div.mm-recipes-details__value')

    if label and value and "Total Time:" in label.get_text(strip=True):
          tiempo = value.get_text(strip=True)
          break # Exit loop once servings are found

  return tiempo

items = soup.find_all('div', class_='mm-recipes-details__item')
tiempo=""
for item in items:
   label = item.find('div', class_='mm-recipes-details__label')
   value = item.find('div', class_='mm-recipes-details__value')

   if label and value and "Total Time:" in label.text:
        tiempo = value.text.strip()
        break # Exit loop once servings are found

print(f"Tiempo total: {tiempo}")

Tiempo total: 1 hr 5 mins


In [14]:
# Factores nutricionales
# En la página existe una lista de factores nutricionales
def obtener_fn (soup):
  items = soup.find_all("span", class_="mm-recipes-nutrition-facts-label__nutrient-name mm-recipes-nutrition-facts-label__nutrient-name--has-postfix")

  return [fact.parent.get_text().replace('\n', ' ').strip() for fact in items]

fn_nutricionales = soup.find_all("span", class_="mm-recipes-nutrition-facts-label__nutrient-name mm-recipes-nutrition-facts-label__nutrient-name--has-postfix")

for i in fn_nutricionales:
  print(i.parent.get_text().replace('\n', ' '))

 Total Fat 23g 
 Saturated Fat 5g 
 Cholesterol 60mg 
 Sodium 1486mg 
 Total Carbohydrate 12g 
 Dietary Fiber 1g 
 Total Sugars 7g 
 Protein 22g 
 Vitamin C 2mg 
 Calcium 21mg 
 Iron 1mg 
 Potassium 232mg 


In [15]:
# Descripción
# La página tiene una descripción de la receta
def obtener_descripcion(soup):
  return soup.find("p", {'class':'article-subheading text-utility-300'}).get_text(strip="True")


resumen = soup.find("p", {'class':'article-subheading text-utility-300'})
resumen.get_text()

"I promise you that this is no gimmick. The spicy, sticky, and peanut-based sauce base delivers a distinct chicken wing-eating experience. If you're a fan of satay, you will enjoy this approach."

In [17]:
# Imagen (Existe varias posibilidades)
# 1) exista imagen: Sacar img principal
print ("Caso 1")
html_content = recuperar_receta("https://www.allrecipes.com/recipe/232863/cinnamon-rubbed-chicken/")
soup = BeautifulSoup(html_content, "html.parser")
item = soup.find("div", class_="primary-image__media")
img = item.find_all("img")
print("Url de la imagen principal:", img[0]["src"])

# 2) exista video: se sacaria el poster del video
print("Caso 2")
html_content = recuperar_receta("https://www.allrecipes.com/recipe/83557/juicy-roasted-chicken/")
soup = BeautifulSoup(html_content, "html.parser")
item = soup.find("div", id="article__primary-video-container_1-0")
img = item.find_all("video")
print("Url de la imagen del poster del video:", img[0]["data-poster"])

# 3) sin imagen: vacío
print("Caso 3")
html_content = recuperar_receta("https://www.allrecipes.com/recipe/266313/fresh-peach-empanadas/")
soup = BeautifulSoup(html_content,"html.parser")

item = soup.find("div", id="article__primary-video-container_1-0")
if len(item.find_all('video')) == 0:
  print("No existe imagen de video principal")

item = soup.find("div", class_="primary-image__media")
if not item:
  print (f"No existe imagen principal")

Caso 1
Url de la imagen principal: https://www.allrecipes.com/thmb/vC0OKMOj_gVP8UBNRvYXz__CRmY=/1500x0/filters:no_upscale():max_bytes(150000):strip_icc()/6696287-20b8752534d84958b7a9594df1cd20ec.jpg
Caso 2
Url de la imagen del poster del video: https://cdn.jwplayer.com/v2/media/EFEVNJX8/thumbnails/brDCeQqa.jpg
Caso 3
No existe imagen de video principal
No existe imagen principal


In [18]:
# se crea función para recuperar el enlace de la imagen, si no existe se envía una cadena vacía
def obtener_url_img(soup):
    video_container = soup.find("div", id="article__primary-video-container_1-0")
    if video_container:
        video = video_container.find("video")
        if video and video.has_attr("data-poster"):
            return video["data-poster"]

    image_container = soup.find("div", class_="primary-image__media")
    if image_container:
        img = image_container.find("img")
        if img and img.has_attr("src"):
            return img["src"]

    return ""

html_content = recuperar_receta("https://www.allrecipes.com/recipe/83557/juicy-roasted-chicken/")
soup = BeautifulSoup(html_content, "html.parser")
print(obtener_url_img(soup))

https://cdn.jwplayer.com/v2/media/EFEVNJX8/thumbnails/brDCeQqa.jpg


In [19]:
# Ingredientes
# En la página existe una lista de ingredientes
def obtener_ingredientes (soup):
  ingredients_section = soup.find_all("li", class_="mm-recipes-structured-ingredients__list-item")
  return [ingrediente.get_text(strip=True) for ingrediente in ingredients_section]


ingredients_section = soup.find_all("li", class_="mm-recipes-structured-ingredients__list-item")
for ingredient in ingredients_section:
  print(ingredient.text.strip())

1 (3 pound) whole chicken, giblets removed
salt and black pepper to taste
1 tablespoon onion powder, or to taste
½ cup butter
1 stalk celery, leaves removed


In [20]:
# Pasos
# En la página existe una lista de pasos para la receta
def obtener_pasos (soup):
  items = soup.find_all("p", class_="comp mntl-sc-block mntl-sc-block-html")
  return [paso.get_text(strip=True) for paso in items]


items = soup.find_all("p", class_="comp mntl-sc-block mntl-sc-block-html")
for i, item in enumerate(items):
  print(f"{i+1}: {item.text}")

1:  Roasted chicken never fails to impress, but it's surprisingly simple to make! This juicy roasted chicken recipe is perfect for beginner cooks and old pros alike.

2:  Roasting a whole chicken at home is easier than it seems. You'll find a detailed ingredient list and step-by-step instructions in the recipe below, but let's go over the basics:

3:  These are the ingredients you'll need to make the juiciest roast chicken recipe of your life:

4:  Here's a very brief overview of what you can expect when you make homemade roasted chicken:

5:  In an oven preheated to 350 degrees F, a 3-pound whole chicken should be completely cooked in a little more than an hour. You'll know it's done when the meat is no longer pink at the bone, the juices run clear, and an instant read thermometer inserted into the thickest part of the thigh (near the bone) reads 165 degrees F.

6:  "Roasting a whole chicken at home requires almost no effort and very few ingredients," says culinary producer Nicole McL

## Parte 3: Obtener enlaces relacionados
* Encontrar links a otras recetas para completar el corpus


In [23]:
# Urls relacionados
# En este caso, se recupera los enlaces de las recetas relacionadas
def obtener_urls_relacionados (soup):
  item = soup.find("div", id="recirc-section_1-0")
  enlaces = item.find_all("a")

  return [enlace["href"] for enlace in enlaces]


item = soup.find("div", id="recirc-section_1-0")
enlaces = item.find_all("a")
for enlace in enlaces:
  print (enlace["href"])

https://www.allrecipes.com/recipe/8709/roasted-lemon-herb-chicken/
https://www.allrecipes.com/recipe/17815/simply-lemon-baked-chicken/
https://www.allrecipes.com/recipe/45954/roast-sticky-chicken-rotisserie-style/
https://www.allrecipes.com/recipe/15427/roast-chicken-with-rosemary/
https://www.allrecipes.com/recipe/9004/marinated-rosemary-chicken/
https://www.allrecipes.com/recipe/8690/spicy-rapid-roast-chicken/
https://www.allrecipes.com/recipe/8596/mediterranean-lemon-chicken/
https://www.allrecipes.com/recipe/15406/honey-baked-chicken-i/
https://www.allrecipes.com/recipe/236306/roasted-chicken-with-lemon-and-rosemary/
https://www.allrecipes.com/recipe/8676/chicken-oreganato/
https://www.allrecipes.com/recipe/8863/ginas-lemon-pepper-chicken/
https://www.allrecipes.com/recipe/217015/grapefruit-chicken/
https://www.allrecipes.com/recipe/228848/stupid-simple-roast-chicken/
https://www.allrecipes.com/recipe/218126/basic-broiled-chicken-breasts/
https://www.allrecipes.com/recipe/189679/ma

## 4. Crear el Scraper

In [24]:
import subprocess
from bs4 import BeautifulSoup

In [25]:
def recuperar_receta(url):
  comando_curl = ["curl", url]
  resultado = subprocess.run(comando_curl, capture_output=True, text=True, check=True)
  return resultado.stdout

In [26]:
html_content = recuperar_receta("https://www.allrecipes.com/recipe/8690/spicy-rapid-roast-chicken/")

In [27]:
def scraping(html_content):
  soup = BeautifulSoup(html_content, "html.parser")

  #scraping
  return {"titulo":obtener_titulo(soup),
          "valoracion": obtener_valoracion(soup),
          "porciones":obtener_porciones(soup),
          "tiempo": obtener_tiempo(soup),
          "descripcion":obtener_descripcion(soup),
          "imagen":obtener_url_img(soup),
          "factores nutricionales": obtener_fn(soup),
          "pasos": obtener_pasos(soup),
          "ingredientes": obtener_ingredientes(soup),
          "links_relacionados": obtener_urls_relacionados(soup)
          }

In [28]:
print(scraping(html_content))

{'titulo': 'Spicy Rapid Roast Chicken', 'valoracion': '4.5', 'porciones': '8', 'tiempo': '1 hr 30 mins', 'descripcion': 'This is the kind of recipe that you just throw together. No need to truss or fuss. Pop it into a very hot oven and it is ready in a hurry.', 'imagen': 'https://cdn.jwplayer.com/v2/media/8yqoTDXW/poster.jpg?width=720', 'factores nutricionales': ['Total Fat 15g', 'Saturated Fat 4g', 'Cholesterol 73mg', 'Sodium 143mg', 'Total Carbohydrate 0g', 'Dietary Fiber 0g', 'Protein 23g', 'Vitamin C 0mg', 'Calcium 14mg', 'Iron 1mg', 'Potassium 207mg'], 'pasos': ['Preheat oven to 450 degrees F (230 degrees C).', 'Rinse chicken thoroughly inside and out under cold running water and remove all fat. Pat dry with paper towels.', 'Put chicken into a small baking pan. Rub with olive oil. Mix the salt, pepper, oregano, basil, paprika and cayenne pepper together and sprinkle over chicken.', 'Roast the chicken in the preheated oven for 20 minutes. Lower the oven to 400 degrees F (205 degree

In [31]:
def web_scraping(url_inicial, limite=100):
    corpus = []
    urls_visitadas = set()
    pendientes = [url_inicial]

    while pendientes and len(corpus) < limite:
        url = pendientes.pop()

        # Para evitar las urls ya visitadas
        if url in urls_visitadas:
            continue

        print(f"Visitando: {url}")

        # Recuperar contenido de la url
        try:
            html_content = recuperar_receta(url)
            scrap = scraping(html_content)
        except Exception as e:
            print(f"Error al procesar {url}: {e}")
            continue

        # Añadir el url a las urls visitadas
        urls_visitadas.add(url)

        # Incorporar nuevos urls
        for nueva_url in scrap.get("links_relacionados", []):
            if nueva_url not in urls_visitadas:
                pendientes.append(nueva_url)

        # Añadir al corpus
        del scrap["links_relacionados"]  # borrar atributo links_relacionados para el corpus
        corpus.append(scrap)

    return corpus

In [32]:
corpus = web_scraping("https://www.allrecipes.com/recipe/83557/juicy-roasted-chicken/")

Visitando: https://www.allrecipes.com/recipe/83557/juicy-roasted-chicken/
Visitando: https://www.allrecipes.com/recipe/232863/cinnamon-rubbed-chicken/
Visitando: https://www.allrecipes.com/recipe/9004/marinated-rosemary-chicken/
Visitando: https://www.allrecipes.com/recipe/9034/rosemary-chicken/
Visitando: https://www.allrecipes.com/recipe/82713/blissful-rosemary-chicken/
Visitando: https://www.allrecipes.com/recipe/8730/oregano-chicken/
Visitando: https://www.allrecipes.com/recipe/8776/cheddar-chicken/
Visitando: https://www.allrecipes.com/recipe/264204/spicy-keto-chicken-and-cheese-casserole/
Visitando: https://www.allrecipes.com/recipe/70522/garlic-cheddar-chicken/
Visitando: https://www.allrecipes.com/recipe/19135/broiled-herb-butter-chicken/
Visitando: https://www.allrecipes.com/recipe/272657/keto-open-faced-chicken-cordon-bleu/
Visitando: https://www.allrecipes.com/recipe/80827/easy-garlic-broiled-chicken/
Visitando: https://www.allrecipes.com/recipe/73108/moist-garlic-chicken/
V

In [51]:
# Crear el DataFrame para el corpus
import pandas as pd
corpus_df = pd.DataFrame(corpus)

corpus_df.head()

,titulo,valoracion,porciones,tiempo,descripcion,imagen,factores nutricionales,pasos,ingredientes
0,Juicy Roasted Chicken,4.8,6,2 hrs,This roasted chicken is perfectly seasoned and...,https://cdn.jwplayer.com/v2/media/EFEVNJX8/thu...,"[Total Fat 32g, Saturated Fat 7g, Cholesterol ...","[Roasted chicken never fails to impress, but i...","[1(3 pound)whole chicken, giblets removed, sal..."
1,Cinnamon-Rubbed Chicken,4.9,4,1 hr 20 mins,I got this idea from a long-ago TV show (Littl...,https://www.allrecipes.com/thmb/vC0OKMOj_gVP8U...,"[Total Fat 31g, Saturated Fat 7g, Cholesterol ...",[Preheat oven to 350 degrees F (175 degrees C)...,"[¼cupolive oil, 1teaspoonground cinnamon, 1tea..."
2,Marinated Rosemary Chicken,4.3,7,,This is a wonderful way to fix your Sunday roa...,https://www.allrecipes.com/thmb/TUlQ4PA08rdPrX...,"[Total Fat 56g, Saturated Fat 11g, Cholesterol...",[To Make Marinade: In a food processor blend t...,"[2(2 to 3 pound)whole chicken, 2bunchesfresh p..."
3,Rosemary Chicken,4.1,4,35 mins,This rosemary chicken breast recipe is baked w...,https://www.allrecipes.com/thmb/NzuKcUPuUIqyQ4...,"[Total Fat 8g, Saturated Fat 1g, Cholesterol 6...",[Preheat oven to 350 degrees F (175 degrees C)...,"[4skinless, boneless chicken breast halves, 2t..."
4,Blissful Rosemary Chicken,4.5,4,40 mins,An elegant and intensely flavorful way to prep...,https://www.allrecipes.com/thmb/fWhUWkVKOCCaXz...,"[Total Fat 30g, Saturated Fat 10g, Cholesterol...",[Use a knife or grater to sharpen the thick en...,"[4sprigsfresh rosemary, 4skinless, boneless ch..."


In [101]:
def fila_a_documento(fila):
    return  f"""
    title: {fila['titulo']}
    raiting: {fila['valoracion']}
    servings: {fila['porciones']}
    total time: {fila['tiempo']}

    description: {fila['descripcion']}

    ingredients: {fila['ingredientes']}

    steps: {fila['pasos']}

    nutricional factors: {fila['factores nutricionales']}
    """

In [102]:
# Generar content para corpus
corpus_df["content"] = corpus_df.apply(lambda x: fila_a_documento(x), axis=1)

In [103]:
corpus_df.head()

,titulo,valoracion,porciones,tiempo,descripcion,imagen,factores nutricionales,pasos,ingredientes,content,preprocessed,embeddings
0,Juicy Roasted Chicken,4.8,6,2 hrs,This roasted chicken is perfectly seasoned and...,https://cdn.jwplayer.com/v2/media/EFEVNJX8/thu...,"[Total Fat 32g, Saturated Fat 7g, Cholesterol ...","[Roasted chicken never fails to impress, but i...","[1(3 pound)whole chicken, giblets removed, sal...",\n title: Juicy Roasted Chicken\n raitin...,tulo juicy roast chicken valoraci n porciones ...,"[-0.00760706, -0.08166299, -0.014807371, -0.01..."
1,Cinnamon-Rubbed Chicken,4.9,4,1 hr 20 mins,I got this idea from a long-ago TV show (Littl...,https://www.allrecipes.com/thmb/vC0OKMOj_gVP8U...,"[Total Fat 31g, Saturated Fat 7g, Cholesterol ...",[Preheat oven to 350 degrees F (175 degrees C)...,"[¼cupolive oil, 1teaspoonground cinnamon, 1tea...",\n title: Cinnamon-Rubbed Chicken\n rait...,tulo cinnamon rub chicken valoraci n porciones...,"[-0.08531587, -0.050379764, -0.022936996, -0.0..."
2,Marinated Rosemary Chicken,4.3,7,,This is a wonderful way to fix your Sunday roa...,https://www.allrecipes.com/thmb/TUlQ4PA08rdPrX...,"[Total Fat 56g, Saturated Fat 11g, Cholesterol...",[To Make Marinade: In a food processor blend t...,"[2(2 to 3 pound)whole chicken, 2bunchesfresh p...",\n title: Marinated Rosemary Chicken\n r...,tulo marinate rosemary chicken valoraci n porc...,"[-0.014826353, -0.05466562, -0.006772316, -0.0..."
3,Rosemary Chicken,4.1,4,35 mins,This rosemary chicken breast recipe is baked w...,https://www.allrecipes.com/thmb/NzuKcUPuUIqyQ4...,"[Total Fat 8g, Saturated Fat 1g, Cholesterol 6...",[Preheat oven to 350 degrees F (175 degrees C)...,"[4skinless, boneless chicken breast halves, 2t...",\n title: Rosemary Chicken\n raiting: 4....,tulo rosemary chicken valoraci n porciones tie...,"[-0.045748133, -0.09574673, -0.023035247, -0.0..."
4,Blissful Rosemary Chicken,4.5,4,40 mins,An elegant and intensely flavorful way to prep...,https://www.allrecipes.com/thmb/fWhUWkVKOCCaXz...,"[Total Fat 30g, Saturated Fat 10g, Cholesterol...",[Use a knife or grater to sharpen the thick en...,"[4sprigsfresh rosemary, 4skinless, boneless ch...",\n title: Blissful Rosemary Chicken\n ra...,tulo blissful rosemary chicken valoraci n porc...,"[-0.018333787, -0.073669456, -0.045043945, -0...."


## Parte 5: Hacer RAG con las recetas obtenidas
* Una vez que se ha construido el corpus, implementar y desplegar RAG para realizar búsquedas en el corpus

#### 5.0. Instalar dependencias

In [ ]:
!pip install sentence-transformers

In [ ]:
!pip install faiss-cpu

In [ ]:
!pip install dotenv

#### 5.1. Preprocesamiento

In [59]:
# Preprocesamiento
import nltk
import re
from nltk.corpus import stopwords, wordnet
from nltk.tokenize import regexp_tokenize
from nltk.tag import pos_tag
from nltk.stem import WordNetLemmatizer

paquetes_requeridos = [
    'punkt','punkt_tab','stopwords', 'wordnet', 'averaged_perceptron_tagger', 'averaged_perceptron_tagger_eng'
]
lemmatizer = WordNetLemmatizer()

# Descargar recursos necesarios
for i in paquetes_requeridos:
  nltk.download(i)

# Lista de stopwords en inglés
stop_words = set(stopwords.words('english'))

def lematizar(tokens):
    tagged = pos_tag(tokens)

    def get_wordnet_pos(tag):
            if tag.startswith('J'):
                return wordnet.ADJ
            elif tag.startswith('V'):
                return wordnet.VERB
            elif tag.startswith('N'):
                return wordnet.NOUN
            elif tag.startswith('R'):
                return wordnet.ADV
            return wordnet.NOUN

    return [
           lemmatizer.lemmatize(word, get_wordnet_pos(pos))
           for word, pos in tagged
        ]

def preprocesar(doc):
    """
    Preprocesa el texto eliminando caracteres especiales, convirtiendo a minúsculas,
    tokenizando, eliminando stopwords, lematizacion
    """
    # Convertir a minúsculas
    doc = doc.lower()

    # normalizar y tokenizar
    words = regexp_tokenize(doc, r"[a-zA-Z]+")

    # stopwords
    for word in stop_words:
      if word in words:
        words.remove(word)

    # lematizar
    words = lematizar(words)

    return ' '.join(words)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger_eng.zip.


In [104]:
corpus_df['preprocessed'] = corpus_df['content'].apply(preprocesar)

#### 5.2. Embeddings

In [105]:
from sentence_transformers import SentenceTransformer
modelo_SBERT = SentenceTransformer('all-MiniLM-L6-v2')

In [106]:
corpus_df["embeddings"] = corpus_df["preprocessed"].apply(modelo_SBERT.encode)

#### 5.3. Indexación en base FAISS

In [107]:
import faiss
import numpy as np
key="embeddings"

In [108]:
embedding_dim = len(corpus_df[key].values[1])
indice = faiss.IndexFlatL2(embedding_dim)

In [109]:
# Añadir vectores al indice
indice.add(np.vstack(corpus_df[key].values).astype("float32")) # faiss usa float32

#### 5.4. Búsqueda y obtención de base

In [110]:
def obtener_base(query):
  query_preprocesada = preprocesar(query)
  query_emb = modelo_SBERT.encode(query_preprocesada)
  _, indices_recuperados = indice.search(query_emb.reshape(1,-1), k=5)
  return corpus_df.loc[indices_recuperados.flatten(), ["content", "imagen"]]

#### 5.5. Generación del Prompt

In [146]:
def generar_prompt(base, query):
    return f"""Eres una aplicación de Retrieval-Augmented Generation (RAG) especializada en recetas de cocina. Siempre respondes en español.

Tu tarea es generar una respuesta clara, completa y escrita en lenguaje natural. Si la información está presente en la base proporcionada, incluye la receta completa (ingredientes, pasos, etc.) y responde `"respuesta_valida": true`.

Si la base no contiene suficiente información para responder la consulta, indica `"respuesta_valida": false` y deja `"respuesta"` con un mensaje explicando que no hay datos disponibles. No inventes contenido.

Devuelve únicamente un JSON con esta estructura:

{{
  "respuesta_valida": true o false,
  "respuesta": "texto de la respuesta aquí"
}}

### Base de conocimientos:
{base}

### Consulta del usuario:
{query}
"""

In [162]:
import json
def procesar_respuesta_json(respuesta):
   respuesta = respuesta.replace("```", "")
   respuesta = respuesta.replace("json", "").strip()
   datos = json.loads(respuesta)
   return [datos["respuesta_valida"], datos["respuesta"]]

#### 5.6. Modelo LLM

In [80]:
from dotenv import load_dotenv
from google import genai
import os

load_dotenv()

client = genai.Client(api_key=os.getenv("GOOGLE_API_KEY"))

def generar_respuesta (prompt):
  response = client.models.generate_content(
    model="gemini-2.5-flash",
    contents=prompt,
  )
  return response.text

In [142]:
# aqui dado que el corpus esta en ingles se adhiere una capa de abstracción para traducirla al idioma inglés
def traducir_query(query):
    return generar_respuesta(
        f"""Traduce la siguiente consulta al inglés, adaptándola para ser utilizada como entrada en un sistema RAG sobre recetas de cocina.
Sintetiza lo irrelevante, conserva la intención culinaria y proporciona solo la traducción final, sin explicaciones ni notas.

Consulta: {query}"""
)

In [167]:
def consultar(query):
  query_traducida = traducir_query(query)
  base = obtener_base(query_traducida)
  prompt = generar_prompt(base["content"].values,query)
  respuesta_consulta = generar_respuesta(prompt)
  estado, respuesta = procesar_respuesta_json(respuesta_consulta)
  img = None
  if estado:
    imgs = base["imagen"].values
    img = imgs[0] if imgs[0] != "" else "https://raspeigindustrial.es/assets/admin/images/default-large.png"

  return respuesta, img

#### 5.6. RAG

##### Ejemplos para buscar

In [125]:
#Busqueda por tiempo y valoracion
corpus_df.loc[[70], ["titulo", "valoracion", "tiempo"]].values

array([['Fresh Fruit Cake', '3.0', '1 hr 40 mins']], dtype=object)

In [124]:
# Busqueda por sus ingredientes
corpus_df.loc[[15], ["titulo", "ingredientes"]].values

array([['Chicken Macaroni Salad',
        list(['2 ½poundsskinless, boneless chicken breast halves', '2cupsmacaroni', '1(15 ounce) canmixed vegetables, drained', '2cupsshredded lettuce', '3cupsmayonnaise', '¼tablespoondried basil', 'salt and pepper to taste', '1pinchgarlic powder'])]],
      dtype=object)

In [127]:
# Busqueda por factores nutricionales
corpus_df.loc[[90], ["titulo", "factores nutricionales"]].values

array([['Gluten-Free Fresh Peach Cobbler',
        list(['Total Fat 15g', 'Saturated Fat 10g', 'Cholesterol 41mg', 'Sodium 244mg', 'Total Carbohydrate 63g', 'Dietary Fiber 2g', 'Total Sugars 33g', 'Protein 4g', 'Vitamin C 37mg', 'Calcium 147mg', 'Iron 1mg', 'Potassium 153mg'])]],
      dtype=object)

##### Ejecución RAG

In [169]:
from IPython.display import Markdown, display, Image

while True:
  query = input(" > Ingrese su consulta: ")
  if query.strip().upper() == 'SALIR':
      print("---- Consulta finalizada-----")
      break

  respuesta, img =  consultar(query)
  if img:
    display(Image(url=img,width=300, height=200))
  display(Markdown(f"**Respuesta:** {respuesta}"))

 > Ingrese su consulta: Juicy Roasted Chicken


**Respuesta:** Aquí tienes la receta de "Juicy Roasted Chicken":

**Título:** Pollo asado jugoso
**Descripción:** Este pollo asado está perfectamente sazonado y cocinado tal como lo hacía mi abuela. ¡El método utilizado en esta receta da como resultado el pollo más jugoso! Nos encantaba picotear el apio después de cocinarlo.
**Calificación:** 4.8 de 5 estrellas
**Porciones:** 6
**Tiempo total:** 2 horas

**Ingredientes:**
*   1 (3 libras) pollo entero, sin menudencias
*   Sal y pimienta negra al gusto
*   1 cucharada de cebolla en polvo, o al gusto
*   ½ taza de mantequilla
*   1 tallo de apio, sin hojas

**Instrucciones:**
1.  Reúne todos los ingredientes. Precalienta el horno a 350 grados F (175 grados C).
2.  Coloca el pollo en una fuente para asar; sazona generosamente por dentro y por fuera con cebolla en polvo, sal y pimienta. Coloca 3 cucharadas de mantequilla en la cavidad del pollo; distribuye el resto de la mantequilla en el exterior del pollo.
3.  Corta el apio en 3 o 4 trozos; colócalos en la cavidad del pollo.
4.  Hornea el pollo sin cubrir en el horno precalentado hasta que ya no esté rosado en el hueso y los jugos salgan claros, aproximadamente 1 hora y 15 minutos. Un termómetro de lectura instantánea insertado en la parte más gruesa del muslo, cerca del hueso, debe marcar 165 grados F (74 grados C).
5.  Retira del horno y rocía con los jugos de cocción. Cubre con papel de aluminio y deja reposar durante unos 30 minutos antes de servir.

 > Ingrese su consulta: dame una receta que tenga fresas


**Respuesta:** Aquí tienes una receta que incluye fresas:

**Mini Tartaletas de Fresa**

**Calificación:** 4.6 de 5 estrellas
**Porciones:** 35
**Tiempo total:** 1 hora 18 minutos

**Descripción:** Estas tartaletas son excelentes para reuniones familiares o fiestas. Toda la dulzura del bizcocho de fresa, pero en porciones pequeñas e individuales, muy convenientes para tus invitados. Este postre de aspecto muy impresionante hará que todos tus invitados pidan la receta.

**Ingredientes:**
* 2 (paquetes de 8 onzas) de queso crema, ablandado
* 2 tazas de mantequilla
* 4 ½ tazas de harina para todo uso
* 3 (paquetes de 3 onzas) de mezcla de gelatina con sabor a fresa Jell-O®
* 1 taza de azúcar blanca
* 3 gotas de colorante rojo para alimentos
* 3 ½ tazas de agua hirviendo
* ¼ taza de maicena
* ¼ taza de agua
* 3 libras de fresas frescas, en rodajas
* 1 ½ tazas de crema batida, o al gusto (Opcional)

**Pasos:**
1. Precalienta el horno a 350 grados F (175 grados C). Engrasa ligeramente los moldes para mini muffins.
2. Coloca el queso crema y la mantequilla en un tazón grande. Bate con una batidora eléctrica hasta que esté suave y esponjoso. Bate gradualmente la harina, una taza a la vez, hasta que esté toda incorporada. Forma 70 bolitas con la masa y presiona cada una en una cavidad del molde para mini muffins para formar las bases de la masa.
3. Hornea las bases en el horno precalentado hasta que estén doradas, de 15 a 18 minutos. Retira del horno y deja enfriar.
4. Incorpora la gelatina, el azúcar y el colorante rojo al agua hirviendo. Coloca a fuego alto; lleva de nuevo a ebullición. Mezcla la maicena y el agua para hacer una pasta. Incorpora la mezcla de maicena en la gelatina hirviendo hasta que se disuelva. Retira del fuego y deja enfriar completamente, unos 30 minutos.
5. Vierte la mezcla de gelatina enfriada de manera uniforme en las bases de las tartaletas. Empuja media fresa en cada tartaleta. Si lo deseas, cubre cada tartaleta con una pequeña cantidad de crema batida o aderezo batido justo antes de servir.

 > Ingrese su consulta: dame una receta alta en hierro y potasio


**Respuesta:** Claro, aquí tienes una receta alta en hierro y potasio:

**Ensalada de Pollo con Fresas y Queso Feta con Aderezo de Fresa-Balsámico Asado**

**Descripción:** Una ensalada de verano maravillosamente fácil, con un aderezo dulce y picante. ¡Te encantarán las fresas asadas!

**Valoración:** 5.0
**Porciones:** 2
**Tiempo total:** 1 hora

**Ingredientes:**
*   1 taza de lechuga romana picada
*   1 taza de espinacas frescas
*   1 taza de col rizada picada
*   1 taza de fresas frescas, sin tallo y en rodajas
*   1 pechuga de pollo cocida, en rodajas
*   ½ taza de almendras en rodajas
*   ¼ taza de queso feta desmenuzado
*   1 taza de fresas frescas, sin tallo y en cuartos (para el aderezo)
*   1 cucharada de azúcar blanca (para el aderezo)
*   ¼ taza de vinagre balsámico (para el aderezo)
*   ¼ taza de aceite de oliva (para el aderezo)
*   1 cucharada de miel (para el aderezo)
*   1 cucharada de mostaza Dijon (para el aderezo)
*   1 diente de ajo, picado (para el aderezo)
*   Sal y pimienta negra molida al gusto (para el aderezo)

**Pasos:**
1.  Mezcla la lechuga romana, las espinacas y la col rizada en un tazón mediano. Distribuye las fresas en rodajas uniformemente sobre las verduras. Coloca el pollo sobre la ensalada. Adorna con las almendras en rodajas y el queso feta.
2.  Precalienta el horno a 175 grados C (350 grados F). Cubre una bandeja para hornear con papel pergamino.
3.  Coloca las fresas en cuartos en la bandeja preparada. Espolvorea con azúcar.
4.  Asa en el horno precalentado hasta que estén blandas y doradas, aproximadamente 20 minutos. Deja enfriar 10 minutos.
5.  Combina las fresas asadas con su jugo, el vinagre balsámico, el aceite de oliva, la miel, la mostaza, el ajo, la sal y la pimienta en un procesador de alimentos; procesa el aderezo hasta que quede suave. Rocía el aderezo sobre la ensalada y sirve.

**Factores nutricionales (por porción):**
*   Grasa total: 50g
*   Grasa saturada: 10g
*   Colesterol: 64mg
*   Sodio: 676mg
*   Carbohidratos totales: 41g
*   Fibra dietética: 6g
*   Azúcares totales: 29g
*   Proteína: 25g
*   Vitamina C: 113mg
*   Calcio: 301mg
*   **Hierro: 3mg**
*   **Potasio: 694mg**

 > Ingrese su consulta: SALIR
---- Consulta finalizada-----
